# What hotel attributes to retain?

In [1]:
# Dependencies
import pandas as pd
import sqlite3

In [2]:
# Connect to the SQLite database
database = "Data/Hotels.db"
conn = sqlite3.connect(database)

# Load data from sqlite3 database
df = pd.read_sql_query("select * from metadata2;", conn)
conn.close()

# Preview the database
df.head()

,index,name,city,province,airportDistance_km,categories2,airport,apartment,attractions,bars,...,reservations,resort,restaurant,services,skiing,spas,theater,utility,village,wedding
0,0,Rancho Valencia Resort Spa,Rancho Santa Fe,CA,14.308848055373579,"['hotels', ',', 'hotels', 'and', 'motels', ','...",0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3,Aloft Arundel Mills,Hanover,MD,4.668331572785505,"['hotels', ',', 'hotels', 'and', 'motels', ','...",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,9,Hampton Inn Suites PortlandVancouver,Vancouver,WA,6.591900084053486,"['hotels', ',', 'hotels', 'and', 'motels', ','...",0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,15,Hotel Phillips,Kansas City,MO,2.6706451419692976,"['hotels', ',', 'caterers', ',', 'hotels', 'an...",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,20,The Inn at Solvang,Huntingdon,PA,3.781816947263244,"['bed', 'breakfasts', ',', 'bed', 'breakfast',...",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# Select quantitative variables
columns = list(df.columns)
df2 = df[columns[6:]]

# Convert all columns from strings to float
df2 = df2.apply(pd.to_numeric)

# Preview the data types
df2.dtypes

airport            float64
apartment          float64
attractions        float64
bars               float64
beach              float64
boutique           float64
breakfast          float64
cabins             float64
cable              float64
campground         float64
casino             float64
cemetery           float64
chalets            float64
chapels            float64
clinics            float64
condominiums       float64
convention         float64
cottages           float64
e-commerce         float64
entertainment      float64
extended           float64
fairgrounds        float64
family-friendly    float64
fashion            float64
fitness            float64
golf               float64
gym                float64
hall               float64
harbor             float64
health             float64
hotel              float64
karaoke            float64
lakeview           float64
lodge              float64
lodges             float64
loft               float64
marina             float64
m